In [1]:
import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = str(pow(2,40))
import cv2 as cv
from matplotlib import pyplot as plt

import onnxruntime as ort
import onnx
import numpy as np
model_path = "models/road_segmentation_model_with_metadata_26_10_22.onnx"
onnx_model = onnx.load(model_path)
onnx.checker.check_model(onnx_model)



In [2]:
input_shapes = [[d.dim_value for d in _input.type.tensor_type.shape.dim] for _input in onnx_model.graph.input]
print("Input Shape:",input_shapes)

arguments = dir(onnx_model)
file = open('arguments.txt','w')
for item in arguments:
    file.write(item)
    file.write('\n')
file.close()

Input Shape: [[1, 3, 512, 512]]


In [3]:
PATH = "Images/581304/07-06.tif"
img = cv.imread(PATH, cv.IMREAD_LOAD_GDAL)
print(img.shape)


(4000, 4000, 3)


In [4]:
#define the priority order for the execution providers
# prefer CUDA Execution Provider over CPU Execution Provider
EP_list = ['CPUExecutionProvider']

# initialize the model.onnx
sess = ort.InferenceSession(model_path, providers = EP_list)





In [7]:
# required_input = [1,3,512,512]

resize_img = cv.resize(img, (512, 512), 
               interpolation = cv.INTER_LINEAR)
print(resize_img.shape)
resize_img =resize_img.reshape([1,3,512,512])
print(resize_img.dtype)
x = resize_img /255.0
x = x.astype(np.float32)
print(x.shape, x.dtype)



# get the outputs metadata as a list of :class:`onnxruntime.NodeArg`
output_name = sess.get_outputs()[0].name
print("Output shape:", output_name)

# get the inputs metadata as a list of :class:`onnxruntime.NodeArg`
input_name = sess.get_inputs()[0].name
print("Input Name:", input_name)

# inference run using image_data as the input to the model 
detections = sess.run([output_name], {input_name: x})[0]

print("Output shape:", detections.shape)

(512, 512, 3)
uint8
(1, 3, 512, 512) float32
Output shape: output
Input Name: input
Output shape: (1, 2, 512, 512)
